In [ ]:
import os

In [ ]:
%pwd

In [ ]:
os.chdir('../')

In [ ]:
from dataclasses import dataclasses
from pathlib import Path


@dataclass(frozen = True)

class ModelTrainerConfig:
    root_dir: Path = Path.cwd()
    train_data_path: Path
    test_data_path: Path
    model_name: str
    max_depth: int
    num_leaves : int
    min_child_samples: int
    learning_rate: float
    n_estimators: int
    min_child_weight: int
    subsample: float
    colsample_bytree: float
    reg_alpha: float
    reg_lambda: float
    random_state: int
    extra_trees: bool
    target_column: str
    

In [ ]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml,create_directory



In [ ]:
class ConfigurationManager:
    def__init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH:

        self.config_filepath = config_filepath
        self.params_filepath = params_filepath
        self.schema_filepath = schema_filepath

        create_directory([self.config_filepath, self.params_filepath, self.schema_filepath])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.LGBMRegressor
        schema=self.schema.TARGET_COLUMN

    
     create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            max_depth=params.max_depth,
            num_leaves=params.num_leaves,
            min_child_samples=params.min_child_samples,
            learning_rate=params.learning_rate,
            n_estimators=params.n_estimators,
            min_child_weight=params.min_child_weight,
            subsample=params.subsample,
            colsample_bytree=params.colsample_bytree,
            reg_alpha=params.reg_alpha,
            reg_lambda=params.reg_lambda,
            random_state=params.random_state,
            extra_trees=params.extra_trees,
            target_column=schema.name
        )

        return model_trainer_config

In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        X_train = train_data.drop([self.config.target_column], axis=1)
        X_test = test_data.drop([self.config.target_column], axis=1)
        y_train = train_data[[self.config.target_column]]
        y_test = test_data[[self.config.target_column]]

        lgbm_reg = LGBMRegressor(max_depth=self.config.max_depth,
                                 num_leaves=self.config.num_leaves,
                                 min_child_samples=self.config.min_child_samples,
                                 learning_rate=self.config.learning_rate,
                                 n_estimators=self.config.n_estimators,
                                 min_child_weight=self.config.min_child_weight,
                                 subsample=self.config.subsample,
                                 colsample_bytree=self.config.colsample_bytree,
                                 reg_alpha=self.config.reg_alpha,
                                 reg_lambda=self.config.reg_lambda,
                                 random_state=self.config.random_state,
                                 extra_trees=self.config.extra_trees
        )

        logger.info("Training model...")

        lgbm_reg.fit(X_train, y_train)

        logger.info("Model trained successfully!")

        joblib.dump(lgbm_reg, os.path.join(self.config.root_dir, self.config.model_name))

        logger.info("Model saved successfully!")

In [ ]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e